# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

## Analysis of GOOG Stock Trends

The following is an analysis of financial data for Google, Inc. (GOOG) based on a formatted HTML table showing weekly prices over a year. The key data points provided include the opening price, closing price, highest and lowest prices for each week, as well as the trading volume.

### Summary of Trends

1. **Stock Price Movement**: Over the provided timeframe, GOOG's stock price showed significant volatility. In early January 2024, the stock opened at $194.36, closing at $196.98 on January 15, 2025. 

2. **Recent Highs and Lows**:
   - The highest recorded price in this period was $202.88 on December 16, 2024.
   - The lowest recorded price was $137.07 on March 11, 2024.

3. **Overall Performance**:
   - The stock price has generally trended upward from its lows in early 2024 to its highs in late 2024. 
   - Following the dip in March 2024, the price steadily recovered, indicating a recovery trend.

4. **Volume Insights**:
   - The trading volume fluctuated significantly. For example, the highest trading volume was over 156 million on December 16, 2024.
   - Higher trading volumes often indicate increased investor interest, which can affect price movement.

5. **Dividends**:
   - There were regular dividends distributed, such as $0.20 on September 9, 2024, and December 9, 2024, which signifies that the company is returning value to shareholders. 

### Percentage Return Calculation

To calculate the total returns over the year, we can use the formula:

\[
\text{Total Return (\%)} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
\]

- **Initial Price** (January 15, 2024): $143.43
- **Final Price** (January 15, 2025): $196.98

Substituting the values into the formula:

\[
\text{Total Return (\%)} = \left( \frac{196.98 - 143.43}{143.43} \right) \times 100 \approx \left( \frac{53.55}{143.43} \right) \times 100 \approx 37.27\%
\]

### Conclusion

In summary, GOOG has experienced a positive upward trend over the analyzed year, marked by considerable price fluctuations and robust trading volumes. The estimated total return stands at **approximately 37.27%**, indicating a profitable investment opportunity during this timeframe. 

Investors should keep an eye on these trends and volume changes, as they can guide future investment strategies.

In [ ]:
display_analysis("PFE")

In [ ]:
display_analysis("AAPL")